In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from math import floor
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


In [7]:
# hyperparameters for tuning: #of neurons, activation function, optimizer, learning rate, batch size, epochs, # of layers
# #of neurons;10-100 # of epochs:20-100
# neurons = [10, 32, 64]
# epochs: [1,5,10]
# batch_size: [500, 1000, 2000]
# activation: ['relu', 'tanh', 'sigmoid']
# optimizer: ['SGD', 'RMSprop', 'Adam']
# learning_rate: [0.001, 0.01, 0.1]
# momentum: [0.1, 0.3, 0.5, 0.7, 0.9]
# drop-out: [0.0, 0.2, 0.4]


In [3]:
from EDA import get_train, get_test
x_train, y_train = get_train()
x_test = get_test()
# select_index = list(range(10))
# x_train = x_train.iloc[:, select_index]
# x_test = x_test.iloc[:, select_index]
# print(x_train.shape, y_train.shape, x_test.shape)
# select_index


(19168, 19)

In [4]:
x_train

,num_beds,num_baths,size_sqft,mean_price,mall_count,property_type_condo,property_type_hdb,property_type_house,tenure_leasehold,built_year_1995,built_year_2005,built_year_2015,built_year_2025,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,school_count_1,school_count_2,mrt_count_1
0,-0.059639,-0.417051,-0.309144,-0.202227,1.0,0,1,0,1,0,0,0,0,0,0,1,0,1,0
1,-0.059639,-0.417051,-0.306586,-0.202227,1.0,0,1,0,1,0,0,0,0,0,0,1,0,1,0
2,-0.059639,-0.417051,-0.306586,-0.202227,1.0,0,1,0,1,0,0,0,0,0,0,1,0,1,0
3,-0.880288,-0.417051,-0.554107,-0.202227,1.0,0,1,0,1,0,0,1,0,0,0,1,0,1,0
4,-0.059639,-0.417051,-0.402524,-0.202227,1.0,1,0,0,1,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19163,1.581659,1.736126,2.815251,-0.148294,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
19164,1.581659,2.453852,2.815251,-0.148294,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
19165,2.402308,3.171578,4.734020,-0.148294,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
19166,1.581659,0.300675,1.088359,-0.192367,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


In [3]:
def define_model(neurons=10, activation='relu', optimizer='SGD', learning_rate=0.1, dropout=0, batch_size=500, epochs=1, init_weights='normal'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=x_train.shape[1], activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))    
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [4]:
seed = 8
np.random.seed(seed)
estimator = KerasRegressor(build_fn=define_model, verbose=0)

In [5]:
# Set paramaters
params_nn ={
    'neurons': [128, 256],
    'activation': ['relu', 'tanh', 'sigmoid'],
    'optimizer': ['SGD', 'RMSprop', 'Adam'],
    'learning_rate': [0.005, 0.01, 0.05, 0.1],
    'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
    'batch_size': [200,500,1000],
    'epochs': [7],
    'init_weights': ['uniform', 'normal', 'he_uniform'],
}
# Run Bayesian Optimization

grid_result = RandomizedSearchCV(
    estimator=estimator, 
    param_distributions=params_nn, 
    n_iter=200,
    cv=5, 
    n_jobs=-1,
    verbose = 10,
    scoring='neg_mean_squared_error'
)

In [6]:
grid_result.fit(x_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


RandomizedSearchCV(cv=5,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000001B72A01D4A8>,
                   n_iter=200, n_jobs=-1,
                   param_distributions={'activation': ['relu', 'tanh',
                                                       'sigmoid'],
                                        'batch_size': [200, 500, 1000],
                                        'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
                                        'epochs': [7],
                                        'init_weights': ['uniform', 'normal',
                                                         'he_uniform'],
                                        'learning_rate': [0.005, 0.01, 0.05,
                                                          0.1],
                                        'neurons': [128, 256],
                                        'optimizer': ['SGD', 'RMSprop',
                                                      'Adam'

In [9]:
y = grid_result.predict(x_test)

In [8]:
grid_result.best_score_

-8.162133691919886e+16

In [11]:
def upload(y):
    submission = pd.read_csv('data/example-submission.csv')
    submission['Predicted'] = y
    submission.to_csv('DNN/submission7.csv', index=False)
    print(submission.head())

In [12]:
upload(y)

   Id      Predicted
0   0  405916.875000
1   1  722270.187500
2   2  265207.968750
3   3  130255.445312
4   4  130392.851562


In [1]:
grid_result.best_params_

NameError: name 'grid_result' is not defined

In [9]:
y

array([1359577.4 , 2060871.4 ,  935751.56, ..., 2851922.5 ,  508567.16,
       2650189.2 ], dtype=float32)